<a href="https://colab.research.google.com/github/feerose111/Learning_PyTorch/blob/main/RNN_based_qa_system_using_pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

df= pd.read_csv('/content/100_Unique_QA_Dataset.csv')

df.head()

,question,answer
0,What is the capital of France?,Paris
1,What is the capital of Germany?,Berlin
2,Who wrote 'To Kill a Mockingbird'?,Harper-Lee
3,What is the largest planet in our solar system?,Jupiter
4,What is the boiling point of water in Celsius?,100


In [2]:
#tokennize

def tokennize(text):
  text = text.lower()
  text = text.replace("?", "")
  text = text.replace("'", "")
  return text.split()

In [3]:
tokennize('	What is the largest planet in our solar system?')

['what', 'is', 'the', 'largest', 'planet', 'in', 'our', 'solar', 'system']

In [4]:
#vocab(dictionary of unique words)
vocab = { "<UNK>": 0 }

In [5]:
def build_vocab(row):
  tokennized_question = tokennize(row['question'])
  tokennized_answer = tokennize(row['answer'])

  merged_tokens = tokennized_question + tokennized_answer

  for token in merged_tokens:
    if token not in vocab:
      vocab[token] = len(vocab)

  return vocab

In [6]:
df.apply(build_vocab, axis=1)

,0
0,"{'<UNK>': 0, 'what': 1, 'is': 2, 'the': 3, 'ca..."
1,"{'<UNK>': 0, 'what': 1, 'is': 2, 'the': 3, 'ca..."
2,"{'<UNK>': 0, 'what': 1, 'is': 2, 'the': 3, 'ca..."
3,"{'<UNK>': 0, 'what': 1, 'is': 2, 'the': 3, 'ca..."
4,"{'<UNK>': 0, 'what': 1, 'is': 2, 'the': 3, 'ca..."
...,...
85,"{'<UNK>': 0, 'what': 1, 'is': 2, 'the': 3, 'ca..."
86,"{'<UNK>': 0, 'what': 1, 'is': 2, 'the': 3, 'ca..."
87,"{'<UNK>': 0, 'what': 1, 'is': 2, 'the': 3, 'ca..."
88,"{'<UNK>': 0, 'what': 1, 'is': 2, 'the': 3, 'ca..."


In [7]:
len(vocab)

324

In [8]:
#text to index
def text_to_index(text, vocab):

  indexed_text = []

  for token in tokennize(text):

    if token in vocab:
      indexed_text.append(vocab[token])
    else:
      indexed_text.append(vocab['<UNK>'])

  return indexed_text


In [9]:
text_to_index('What is your name?', vocab)

[1, 2, 0, 0]

In [10]:
#dataset class
import torch
from torch.utils.data import Dataset,DataLoader

class QADataset(Dataset):
  def __init__(self, df, vocab):
    self.df = df
    self.vocab = vocab

  def __len__(self):
    return len(self.df)

  def __getitem__(self, index):
    numbered_ques = text_to_index(self.df.iloc[index]['question'], self.vocab)
    numbered_ans = text_to_index(self.df.iloc[index]['answer'], self.vocab)

    return torch.tensor(numbered_ques), torch.tensor(numbered_ans)

In [11]:
dataset = QADataset(df, vocab)

In [12]:
dataset[0]

(tensor([1, 2, 3, 4, 5, 6]), tensor([7]))

In [13]:
dataloader = DataLoader(dataset, batch_size=1, shuffle=True)

In [14]:
for question, answer in dataloader:
  print(question, answer)

tensor([[42, 43, 44, 45, 46, 47, 48]]) tensor([[49]])
tensor([[ 42, 101,   2,   3,  17]]) tensor([[102]])
tensor([[  1,   2,   3, 180, 181, 182, 183]]) tensor([[184]])
tensor([[  1,   2,   3,   4,   5, 113]]) tensor([[114]])
tensor([[  1,   2,   3,  92, 137,  19,   3,  45]]) tensor([[185]])
tensor([[  1,   2,   3,   4,   5, 109]]) tensor([[317]])
tensor([[ 42, 263, 264,  14, 265, 266, 158, 267]]) tensor([[268]])
tensor([[ 10,  96,   3, 104, 239]]) tensor([[240]])
tensor([[  1,   2,   3,  33,  34,   5, 245]]) tensor([[246]])
tensor([[  1,   2,   3,  37,  38,  39, 161]]) tensor([[162]])
tensor([[ 78,  79, 261, 151,  14, 262, 153]]) tensor([[36]])
tensor([[ 42, 137, 118,   3, 247,   5, 248]]) tensor([[249]])
tensor([[ 42, 299, 300, 118,  14, 301, 302, 158, 303, 304, 305, 306]]) tensor([[307]])
tensor([[ 1,  2,  3, 24, 25,  5, 26, 19, 27]]) tensor([[28]])
tensor([[ 78,  79, 129,  81,  19,   3,  21,  22]]) tensor([[36]])
tensor([[ 42, 200,   2,  14, 201, 202, 203, 204]]) tensor([[205]])
ten

RNN Architecture

50 neurons(input layer) | |- > 64 neurons (hidden layer) | |- > 324 neurons (output layer) each neuron for word in vocab

In [15]:
import torch.nn as nn



In [28]:
class SimpleRNN(nn.Module):

  def __init__(self, vocab_size):
    super().__init__()
    self.Embedding = nn.Embedding(vocab_size, 50)
    self.RNN = nn.RNN(50, 64, batch_first = True)
    self.FC = nn.Linear(64, vocab_size)

  def forward(self, question):
    embedded_question = self.Embedding(question)
    hidden, final = self.RNN(embedded_question)
    output = self.FC(final.squeeze(0))

    return output


In [29]:
learning_rate = 0.001
epochs = 20

In [30]:
model = SimpleRNN(len(vocab))

In [31]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr= learning_rate)

In [36]:
#training loop

for epoch in range(epochs):
  total_loss = 0

  for question , answer in dataloader:

    optimizer.zero_grad()

    #forward pass
    prediction = model(question)
    print(prediction.shape)
    print(answer.shape)
    #loss
    loss = criterion(prediction, answer[0])

    #gradient
    loss.backward()

    #uodate
    optimizer.step()

    total_loss += loss.item()

    print(f"Epoch :--> {epoch}, Loss: {total_loss:4f}")

Streaming output truncated to the last 5000 lines.
torch.Size([1, 1])
Epoch :--> 1, Loss: 226.357708
torch.Size([1, 324])
torch.Size([1, 1])
Epoch :--> 1, Loss: 230.692890
torch.Size([1, 324])
torch.Size([1, 1])
Epoch :--> 1, Loss: 236.011857
torch.Size([1, 324])
torch.Size([1, 1])
Epoch :--> 1, Loss: 240.532281
torch.Size([1, 324])
torch.Size([1, 1])
Epoch :--> 1, Loss: 245.196216
torch.Size([1, 324])
torch.Size([1, 1])
Epoch :--> 1, Loss: 250.327704
torch.Size([1, 324])
torch.Size([1, 1])
Epoch :--> 1, Loss: 255.950767
torch.Size([1, 324])
torch.Size([1, 1])
Epoch :--> 1, Loss: 261.155128
torch.Size([1, 324])
torch.Size([1, 1])
Epoch :--> 1, Loss: 266.374043
torch.Size([1, 324])
torch.Size([1, 1])
Epoch :--> 1, Loss: 271.271372
torch.Size([1, 324])
torch.Size([1, 1])
Epoch :--> 1, Loss: 276.614872
torch.Size([1, 324])
torch.Size([1, 1])
Epoch :--> 1, Loss: 281.617876
torch.Size([1, 324])
torch.Size([1, 1])
Epoch :--> 1, Loss: 286.613080
torch.Size([1, 324])
torch.Size([1, 1])
Epoch :

In [42]:
def predict(model, question , threshold=0.5):

  #convert question to number
  question_tensor = torch.tensor(text_to_index(question, vocab)).unsqueeze(0)

  #forward pass
  prediction = model(question_tensor)

  #convert logits to probabilities
  prob = torch.nn.functional.softmax(prediction, dim= 1)

  value , index = torch.max(prob, dim = 1)

  if value < threshold:
    print(" I don't know")
  else:
    print(list(vocab.keys())[index])

In [43]:
predict(model , "what is your name?")

 I don't know
